In [1]:
# Much of the following is created with OpenAI ChatGPT


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


# Specify the file path
file_path = 'TNG50-1-moi-final-v1-67-75-25.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Now you can work with the DataFrame 'df'



In [2]:
print( df.columns )


Index(['Unnamed: 0', 'moi_SubhaloIDRaw', 'snapnum', 'p_acceleration',
       'p_SubhaloIDRaw', 's_SubhaloIDRaw', 'p_SubhaloMass', 's_SubhaloMass',
       'p_SubhaloPos', 's_SubhaloPos', 'p_SubhaloVel', 's_SubhaloVel',
       'p_SubhaloSpin', 's_SubhaloSpin', 'p_SubhaloHalfmassRad',
       's_SubhaloHalfmassRad', 'p_vLink', 's_vLink'],
      dtype='object')


In [3]:
if False:
    # Importing necessary libraries
    import numpy as np
    import matplotlib.pyplot as plt

    # Graph values
    graph_col = ['p_acceleration',
                 'p_SubhaloMass', 's_SubhaloMass',
                 'p_SubhaloPos', 's_SubhaloPos', 'p_SubhaloVel', 's_SubhaloVel',
                 'p_SubhaloSpin', 's_SubhaloSpin', 'p_SubhaloHalfmassRad',
                 's_SubhaloHalfmassRad']

    # Hardcoded snapnum value
    image_snapshot = 67

    # Get unique values in the column 'moi_SubhaloIDRaw'
    unique_values = df['moi_SubhaloIDRaw'].unique()

    # Loop through the unique values
    for value in unique_values:
        # Filter the DataFrame for rows matching the current unique value
        filtered_df = df[df['moi_SubhaloIDRaw'] == value]

        # Create a separate plot for each component (X, Y, Z)
        for col in graph_col:

            # Skip primary galaxy, graph secondary values
            if 'p_' in col:
                continue

            print(col)
            # Check if the column contains x, y, z values
            if 'Pos' in col or 'Vel' in col or 'Spin' in col:

                # Convert the list of x, y, z values into a 2D array of floats
                components = filtered_df[col].values
                xyz_values = np.zeros((len(components), 3))

                for i, r in enumerate(components):
                    xyz_values[i, :] = np.array(r[1:-1].split()).astype(float)

                # Plot each component separately
                fig, ax = plt.subplots()
                c_label = ['x', 'y', 'z' ]
                for i in range(3):
                    ax.plot(filtered_df['snapnum'], xyz_values[:, i], label=f"{col}_{c_label[i]}")

                # Add dashed vertical line at image_snapshot
                ax.axvline(x=image_snapshot, color='k', linestyle='--', label='image_snapshot')

            else:
                # Plot the column directly
                fig, ax = plt.subplots()
                ax.plot(filtered_df['snapnum'], filtered_df[col], label=col)

                # Add dashed vertical line at image_snapshot
                ax.axvline(x=image_snapshot, color='k', linestyle='--', label='image_snapshot')

            # Set plot title and labels
            ax.set_title(f'Graphs over Time - {col} - SubhaloID: {value}')
            ax.set_xlabel('snapnum')
            ax.set_ylabel('Value')

            # Set x-axis ticks as integers
            ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))

            # Add grid
            ax.grid(True)

            # Add legend
            ax.legend()

            # Show the plot
            plt.show()

        # Remove the break statement if not needed
        break

In [4]:
print( df.columns)

Index(['Unnamed: 0', 'moi_SubhaloIDRaw', 'snapnum', 'p_acceleration',
       'p_SubhaloIDRaw', 's_SubhaloIDRaw', 'p_SubhaloMass', 's_SubhaloMass',
       'p_SubhaloPos', 's_SubhaloPos', 'p_SubhaloVel', 's_SubhaloVel',
       'p_SubhaloSpin', 's_SubhaloSpin', 'p_SubhaloHalfmassRad',
       's_SubhaloHalfmassRad', 'p_vLink', 's_vLink'],
      dtype='object')


In [ ]:
# Creating a dictionary to store all the SPAM units.  (From SPAM fortran source code)
spam_units = {
    "mass_gm": 1.98892e44,  # gm  (Assumed SPAM's mass unit, 10^11 M_⊙ )
    "mass_solar": 1.98892e33,  # gm
    "distance": 4.6285203749999994e22,  # cm  (Assumed SPAM's distance unit, roughly 15 kpc)
    "time_s": 2.733342473337471e15,  # s
    "vel_unit": None,  # cm/s, to be calculated dynamically  (Assumed SPAM's velocity unit, roughly 170 km/s)
    "pc": 3.08568025e18,  # cm (parsec)
    "kpc": None,  # kiloparsec, to be calculated dynamically
    "year": 365.25 * 24 * 3600,  # seconds in a year
    "km": 1e5,  # cm
    "vel_km_sec": None,  # velocity in km/s, to be calculated dynamically
    "a_mss": None,  # Acceleration unit, to be calculated dynamically
    "a0_mks": 1.2e-10,  # Acceleration in m/s^2 (used in Modified Newtonian Dynamics)
    "a0": None,  # Normalized acceleration unit, to be calculated dynamically
    "pi": 3.141592653589793  # Pi
}

# Calculating dynamic values based on other constants
spam_units["vel_unit"] = spam_units["distance"] / spam_units["time_s"]
spam_units["kpc"] = spam_units["pc"] * 1000
spam_units["vel_km_sec"] = spam_units["vel_unit"] / spam_units["km"]
spam_units["a_mss"] = spam_units["distance"] / (spam_units["time_s"] ** 2) / 100.0
spam_units["a0"] = spam_units["a0_mks"] / spam_units["a_mss"]

# Dictionary of units for specific fields in the IllustrisTNG data
tng_units = {
    'SubhaloMass': '10^10 M⊙/h',  # Total mass of all bound particles/cells
    'SubhaloPos': 'ckpc/h',       # Comoving kiloparsecs per hubble parameter
    'SubhaloVel': 'km/s',         # Kilometers per second
    'SubhaloSpin': '(kpc/h)(km/s)', # Product of comoving kiloparsecs per hubble parameter and kilometers per second
    'SubhaloHalfmassRad': 'ckpc/h'  # Comoving kiloparsecs per hubble parameter containing half mass
}

def convert_illustris_to_spam(subhalo_data, current_snapnum, past_snapnum, h=0.7):
    """
    Convert IllustrisTNG data units to SPAM units.   
    Grabs data from two different snapshots due to how Subhalo particle data is postprocessed.

    Parameters:
    - subhalo_data: dict, containing subhalo data from IllustrisTNG with specific keys.
    - current_snapnum: snapshot number to pull position and velocity data from.
    - past_snapnum: snapshot number to pull mass, spin, radius
    - h: float, the Hubble constant used in the IllustrisTNG simulation (dimensionless, usually ~0.7).

    Returns:
    - numpy array, converted subhalo data in SPAM units.
    spam_param[0]: X-coordinate of the secondary galaxy's position
    spam_param[1]: Y-coordinate of the secondary galaxy's position
    spam_param[2]: Z-coordinate of the secondary galaxy's position
    spam_param[3]: X-component of the secondary galaxy's velocity
    spam_param[4]: Y-component of the secondary galaxy's velocity
    spam_param[5]: Z-component of the secondary galaxy's velocity
    spam_param[6]: Mass of the primary galaxy. Often set to a unit mass or normalized in simulations
    spam_param[7]: Mass of the secondary galaxy, relative to the primary
    spam_param[8]: Outer radius of the primary galaxy's influence or disk
    spam_param[9]: Outer radius of the secondary galaxy's influence or disk
    spam_param[10]: Azimuthal angle for the primary galaxy
    spam_param[11]: Azimuthal angle for the secondary galaxy
    spam_param[12]: Inclination angle for the primary galaxy
    spam_param[13]: Inclination angle for the secondary galaxy
    spam_param[14]: Softening length for gravitational calculations involving the primary galaxy
    spam_param[15]: Softening length for the secondary galaxy
    spam_param[16]: Scaling factor for radial distribution in the primary galaxy (bulge)
    spam_param[17]: Scaling factor for radial distribution in the primary galaxy (disk)
    spam_param[18]: Scaling factor for radial distribution in the primary galaxy (halo)
    spam_param[19]: Scaling factor for radial distribution in the secondary galaxy (bulge)
    spam_param[20]: Scaling factor for radial distribution in the secondary galaxy (disk)
    spam_param[21]: Scaling factor for radial distribution in the secondary galaxy (halo) 
    """
    
    # Initialize spam data array
    spam_data = np.zeros(22)

    # Grab needed data from subhalo-data
    current_data = subhalo_data[subhalo_data['snapnum'] == current_snapnum]
    past_data = subhalo_data[subhalo_data['snapnum'] == past_snapnum]

    # print( current_data )
    # print( past_data )
    
    
    # Converting string array to array
    def get_ar(s):
        
        # Parse DF thing to string
        s = s.values[0]
        
        # Strip surrounding brackets
        s = s[1:-1]
        
        # Split string and convert to a numpy array of floats
        return np.array(s.split()).astype(float)

    
    # Conversion from IllustrisTNG to SPAM units
    # position
    tmp_pos =  get_ar( current_data['s_SubhaloPos'] ) -  get_ar( current_data['p_SubhaloPos'] )  # Get difference in position
    tmp_pos = tmp_pos * h  # from ckpc/h to kpc
    tmp_pos *= spam_units["kpc"]   # from kpc to cm
    tmp_pos /= spam_units["distance"] # SPAM's standardized distance (roughly 15 kpc)
    spam_data[0:3] = tmp_pos
    
    # velocity
    tmp_vel = get_ar(current_data['s_SubhaloVel']) - get_ar(current_data['p_SubhaloVel']) # Get difference in velocity, in km/s
    tmp_vel *= 1e5  # Convert km/s to cm/s
    tmp_vel = tmp_vel / spam_units['vel_unit'] # Convert cm/s to SPAM's standardized velocity unit
    spam_data[3:6] = tmp_vel 
    
    # [...]
    # velocity
    tmp_vel = get_ar(subhalo_data['s_SubhaloVel']) - get_ar(subhalo_data['p_SubhaloVel']) # Get difference in velocity, in km/s
    tmp_vel *= 1e5  # Convert km/s to cm/s
    spam_data[3:6] = tmp_vel / spam_units['vel_unit'] # Convert cm/s to SPAM's standardized velocity unit
    # [...]

    
    # Masses
    tmp_mass = np.zeros(2)
    tmp_mass[0] = past_data['p_SubhaloMass'].values[0]
    tmp_mass[1] = past_data['s_SubhaloMass'].values[0]
    tmp_mass *=  1e10 / h  # Convert mass from 10^10 M_⊙/h to M_⊙
    tmp_mass *=  spam_units['mass_solar'] / spam_units['mass_gm']  # Convert from M_⊙ to SPAM's mass unit (10^11 M_⊙ )
    spam_data[6:8] = tmp_mass

    # Radius
    # NOTE:  SPAM expects an outer radius.  Illustris pulled a Half Mass Radius.  Modify later 
    tmp_rad = np.zeros(2)
    tmp_rad[0] =  past_data['p_SubhaloHalfmassRad'].values[0]
    tmp_rad[1] =  past_data['s_SubhaloHalfmassRad'].values[0]
    tmp_rad *= h  # from ckpc/h to kpc
    tmp_rad *= spam_units["kpc"]   # from kpc to cm
    tmp_rad /= spam_units["distance"] # SPAM's standardized distance (roughly 15 kpc)
    spam_data[8:10] = tmp_rad
    spam_data[14:16] = tmp_rad / 3 # From JSPAM paper.  Softening length is typically 0.3 times outer radius
    
    # SPIN / Orientation
    # NOTE: SPAM expects two angles per galaxy to describe orientation in reference to the projected viewing direction
    # NOTE:  IllustrisTNG provides x,y,z spin values for each galaxy
    # Therefore:  Assumee reference plane is XY plane. Can be modified later as needed.

    def calculate_orientation_angles( spin ):
        """
        Calculate the azimuthal and inclination angles from the spin components.

        Parameters:
        spin (float array): [ x, y, z ] components of the spin.

        Returns:
        tuple: (phi, theta) where phi is the azimuthal angle and theta is the inclination angle,
               both in radians.
        """
        x, y, z = spin
        # Calculate the magnitude of the vector
        r = np.sqrt(x**2 + y**2 + z**2)

        # Calculate the azimuthal angle in radians
        phi = np.arctan2(y, x)  # This returns the angle in the range [-pi, pi]

        # Calculate the inclination angle in radians
        if r == 0:
            theta = 0  # Undefined direction for a zero vector, set arbitrarily to 0
        else:
            theta = np.arccos(z / r)

        return np.degrees(phi), np.degrees(theta)

    spam_data[10], spam_data[12] = calculate_orientation_angles( get_ar( past_data['p_SubhaloSpin'] ))
    spam_data[11], spam_data[13] = calculate_orientation_angles( get_ar( past_data['s_SubhaloSpin'] ))
        
    return spam_data

if True:
# Get unique moi in the column 'moi_SubhaloIDRaw'
    unique_moi = df['moi_SubhaloIDRaw'].unique()

    # Loop through the unique values
    for moi in unique_moi:
        # Filter the DataFrame for rows matching the current unique value
        df_moi = df[df['moi_SubhaloIDRaw'] == moi]
        spam_moi = convert_illustris_to_spam( df_moi, 67, 57 )
        print( spam_moi )
        break

In [6]:
# Correct the calculation using the given SPAM units

# Constants
distance_cm = 4.6285203749999994e22  # cm (assumed to be roughly 15 kpc)
time_s = 2.733342473337471e15  # s
km_to_cm = 1e5  # conversion factor from km to cm

# Calculate the velocity unit in cm/s
vel_unit_cm_per_s = distance_cm / time_s

# Convert this velocity to km/s for a galactic scale interpretation
vel_unit_km_per_s = vel_unit_cm_per_s / km_to_cm

vel_unit_km_per_s


169.33554503868936